In [43]:
import pandas as pd
import glob
import numpy as np
from itertools import chain
import os
import cv2
from moviepy.editor import VideoFileClip

def get_fps(videoname):
    clip = VideoFileClip(videoname)
    return clip.fps

def read_video(video):
    videogen = skvideo.io.vreader(video)
    new_videogen = itertools.islice(videogen, 0, 1, 1)
    for image in new_videogen:
        a = 1
    return image

def get_video_information(file_path):
    videofiles = np.array(glob.glob(os.path.join(file_path,'video*')))
    videofiles = videofiles[np.array([len(os.path.basename(i)) if i[-3:]!='pkl' else 0 for i in videofiles])==len('video_000000.mp4')]
    # get fps and screen dim
    df_fps = pd.DataFrame()
    fpsl = []
    rowlist = []; collist = []
    for ivideo in videofiles:
        print(ivideo)
        if os.path.basename(ivideo)[-3:]=='avi':
            fps = 30
        else:
            fps = get_fps(ivideo)
        fpsl.append(fps)
        img = read_video(ivideo)
        nrows = len(img)
        ncols = len(img[0])
        rowlist.append(nrows); collist.append(ncols);

    df_fps['fps'] = pd.Series(fpsl)
    df_fps['pixel_x'] = pd.Series(collist)
    df_fps['pixel_y'] = pd.Series(rowlist)
    df_fps['video'] = [os.path.basename(i)[:-4] for i in videofiles]
    return df_fps

def load_raw_pkl_files(path):
    videofiles = np.array(glob.glob(os.path.join(path,'video*')))
    videofiles = videofiles[np.array([len(os.path.basename(i)) if i[-3:]!='pkl' else 0 for i in videofiles])==len('video_000000.mp4')]
    pklfiles = [ivid[:-4]+'.pkl' for ivid in videofiles]
    df_pkl = pd.DataFrame()
    for file in pklfiles:
        one_file = pd.read_pickle(file).reset_index().drop('index',axis = 1)
        df_pkl = df_pkl.append(one_file)
    df_pkl = df_pkl.reset_index().drop('index', axis = 1)
    return df_pkl

def get_skel(df):
    if len(list(itertools.chain(*df.limbs_subset)))>0:
        peaks = df.peaks.iloc[0]
        parts_in_skel = df.limbs_subset.iloc[0]
        person_to_peak_mapping = [list(i[:-2]) for i in parts_in_skel] 
        skel_idx = [[i]*(len(iskel)-2) for i, iskel in enumerate(parts_in_skel)]
        idx_df = pd.DataFrame.from_dict({'peak_idx':list(itertools.chain(*person_to_peak_mapping)),\
         'person_idx':list(itertools.chain(*skel_idx))})
        peaks_list = list(chain.from_iterable(peaks))
        x = [ipeak[0] for ipeak in peaks_list]
        y = [ipeak[1] for ipeak in peaks_list]
        c = [ipeak[2] for ipeak in peaks_list]
        peak_idx = [ipeak[3] for ipeak in peaks_list]
        kp_idx = list(chain.from_iterable([len(ipeak)*[i] for i,ipeak in enumerate(peaks)]))
        peak_df = pd.DataFrame.from_dict({'x':x,'y':y,'c':c,'peak_idx':peak_idx,'part_idx':kp_idx})
        kp_df = pd.merge(idx_df, peak_df, on='peak_idx', how='left').drop('peak_idx',axis=1)
        kp_df = kp_df.loc[~kp_df.c.isnull(),:]
    else:
        kp_df = pd.DataFrame()
    return kp_df
    

In [ ]:
# paths
raw_pose_estimates_video_path = '/data2/clairec/openpose_output/YT'
pose_estimates_path = '../data/pose_estimates/youtube/py'
pose_estimate_animation_path = '../data/pose_estimates/youtube/video'

# get video information
if os.path.exists(os.path.join(pose_estimates_path, 'youtube_video_info.pkl'))==0:
    df_fps = get_video_information(raw_pose_estimates_video_path)
    df_fps.to_pickle(os.path.join(pose_estimates_path, 'youtube_video_info.pkl'))
else:
    df_fps = pd.read_pickle(os.path.join(pose_estimates_path, 'youtube_video_info.pkl')) 

# load raw pkl files
if os.path.exists(os.path.join(pose_estimates_path, 'raw_pose_estimates.pkl'))==0:
    df_pkl = load_raw_pkl_files(raw_pose_estimates_video_path)
    df_pkl.to_pickle(os.path.join(pose_estimates_path, 'raw_pose_estimates.pkl'))
else:
    df_pkl = pd.read_pickle(os.path.join(pose_estimates_path, 'raw_pose_estimates.pkl'))

# process raw pose estimates 
if os.path.exists(os.path.join(pose_estimates_path, 'pose_estimates.pkl'))==0:
    df = df_pkl.groupby(['video', 'frame']).apply(get_skel)
    
    # keep person index with max number of keypoints per frame
    counts = df.groupby(['video','frame', 'person_idx'])['c'].count().reset_index()
    max_rows = counts.groupby(['video','frame'])['c'].idxmax().tolist()
    max_rows_df = counts.loc[max_rows,['video','frame', 'person_idx']]
    max_rows_df['dum'] = 1
    df = pd.merge(df, max_rows_df, on=['video','frame', 'person_idx'], how='inner')

    # add keypoint labels
    bps = ["Nose","Neck","RShoulder","RElbow","RWrist","LShoulder","LElbow","LWrist","RHip","RKnee","RAnkle","LHip","LKnee",\
     "LAnkle","REye","LEye","REar","LEar"]
    df['bp'] = [bps[int(i)] for i in df.part_idx]
    df = df[['video','frame', 'x', 'y', 'bp']] 

    # include row for each keypoint and frame
    max_frame = df.groupby('video').frame.max().reset_index()
    max_frame['frame_vec'] = max_frame.frame.apply(lambda x: np.arange(0,x+1))
    max_frame['bp'] = pd.Series([bps]*len(max_frame))
    y =[]
    _ = max_frame.apply(lambda x: [y.append([x['video'], x['bp'], i]) for i in x.frame_vec], axis=1)
    all_frames = pd.DataFrame(y, columns = ['video','bp','frame'])
    z =[]
    _ = all_frames.apply(lambda x: [z.append([x['video'], x['frame'], i]) for i in x.bp], axis=1)
    all_frames = pd.DataFrame(z, columns = ['video','frame', 'bp'])
    df = pd.merge(df, all_frames, on = ['video','frame','bp'], how='outer')

    df['infant'] = df.video.str[6:12].astype(int)
    df = pd.merge(df,df_fps, on = 'video', how='outer')
    df['time'] = df['frame']/df['fps']
    df.to_pickle(os.path.join(pose_estimates_path, 'pose_estimates.pkl'))

In [103]:

# video 11 and video 11 (1)


# new script
# median and mean filter
# rotating etc
df.video.unique()

array(['video_000000', 'video_000001', 'video_000004', 'video_000005',
       'video_000011 (1)', 'video_000021', 'video_000031', 'video_000047',
       'video_000052', 'video_000059', 'video_000068', 'video_000070',
       'video_000071', 'video_000072', 'video_000073', 'video_000077',
       'video_000079', 'video_000086', 'video_000088', 'video_000089',
       'video_000090', 'video_000099', 'video_000106', 'video_000107',
       'video_000110', 'video_000111', 'video_000112', 'video_000121',
       'video_000122', 'video_000123', 'video_000127', 'video_000135',
       'video_000140', 'video_000141', 'video_000169', 'video_000172',
       'video_000173', 'video_000179', 'video_000186', 'video_000190',
       'video_000191', 'video_000204', 'video_000227', 'video_000236',
       'video_000241', 'video_000244', 'video_000267', 'video_000268',
       'video_000269', 'video_000274', 'video_000275', 'video_000276',
       'video_000282', 'video_000283', 'video_000284', 'video_000285',
  